In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

class EEGNetTransformer(nn.Module):
    def __init__(self, in_channels, num_classes, num_layers, hidden_dim):
        super(EEGNetTransformer, self).__init__()
        self.input_conv = nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)
        self.conv_block1 = ConvBlock(64, 128)
        self.conv_block2 = ConvBlock(128, 256)
        self.transformer = nn.Transformer(d_model=256, nhead=8, num_encoder_layers=num_layers, dim_feedforward=hidden_dim)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, src):
        out = self.input_conv(src)
        out = self.conv_block1(out)
        out = self.conv_block2(out)
        out = out.permute(0, 2, 1)  # 调整维度顺序
        out = self.transformer(out, out)  # 将out作为源数据和目标数据传递给Transformer
        out = out.permute(0, 2, 1)  # 还原维度顺序
        out = torch.mean(out, dim=2)  # Global average pooling
        out = self.fc(out)
        return out

# 创建模型实例
model = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)



# 创建一个示例网络
in_channels = 1  # 输入信号的通道数
num_classes = 2  # 分类的类别数
num_layers = 2  # Transformer中的层数
hidden_dim = 512  # 前馈神经网络的隐藏层维度
eegnet_transformer = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)

# 打印网络结构
print(eegnet_transformer)


EEGNetTransformer(
  (input_conv): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv_block1): ConvBlock(
    (conv): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv_block2): ConvBlock(
    (conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256,

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)  # 从CSV文件中读取数据
        self.labels = self.data.pop('class')  # 将标签列从数据中分离

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data.iloc[index].values  # 获取样本数据
        label = self.labels.iloc[index]  # 获取对应的标签
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


# 创建数据集实例
dataset = CustomDataset("database.csv")

# 划分训练集和验证集
train_dataset, valid_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# 创建一个示例网络
in_channels = 1  # 输入信号的通道数
num_classes = 2  # 分类的类别数
num_layers = 2  # Transformer中的层数
hidden_dim = 512  # 前馈神经网络的隐藏层维度
eegnet_transformer = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)



/tmp/ipykernel_287049/3642991768.py:22: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


NameError: name 'train_daatset' is not defined

In [ ]:
def train_model(model, train_loader, valid_loader, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # 选择优化器
    criterion = nn.CrossEntropyLoss()  # 选择损失函数

    train_losses = []  # 保存训练损失
    valid_losses = []  # 保存验证损失
    valid_accuracies = []  # 保存验证准确率

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for data, labels in train_loader:
            data = data.to(device)
            labels = labels.to(device)
            labels = labels.long()
            optimizer.zero_grad()
            outputs = model(data.unsqueeze(1))  # 添加一维通道
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 计算验证集损失和准确率
        model.eval()
        valid_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for data, labels in valid_loader:
                data = data.to(device)
                labels = labels.to(device)
                outputs = model(data.unsqueeze(1))  # 添加一维通道
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 计算平均损失和准确率
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        valid_accuracy = correct / total

        # 保存训练和验证损失以及验证准确率
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)

        # 打印训练过程中的损失和准确率
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Valid Loss: {valid_loss:.4f} - Valid Accuracy: {valid_accuracy:.4f}")

    # 绘制训练和验证损失曲线
    plt.plot(range(1, num_epochs+1), train_losses, label="Train Loss")
    plt.plot(range(1, num_epochs+1), valid_losses, label="Valid Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

    # 绘制验证准确率曲线
    plt.plot(range(1, num_epochs+1), valid_accuracies)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Validation Accuracy")
    plt.show()

# 训练模型
num_epochs = 10
train_model(eegnet_transformer, train_loader, valid_loader, num_epochs)


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, f1_score
import numpy as np

# 读取数据集
class EEGDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1]
        # 调整数据维度顺序以匹配Conv1d的输入要求
        features = features.reshape(1, -1)  # 从 [sequence_length, in_channels] 调整为 [in_channels, sequence_length]
        return torch.Tensor(features), label




# 定义超参数
in_channels = 1
num_classes = 2
num_layers = 4
hidden_dim = 512
batch_size = 16
num_epochs = 80
learning_rate = 0.001

# 创建数据加载器
dataset = EEGDataset('database.csv')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型实例
model = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(dataloader)
for epoch in range(num_epochs):
    model.train()  # 确保模型处于训练模式
    progress_bar = tqdm(enumerate(dataloader), total=total_step, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    
    # 初始化用于计算评估指标的列表
    all_predictions = []
    all_labels = []
    
    for i, (data, labels) in progress_bar:
        # 前向传播
        outputs = model(data)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'Loss': loss.item()})
    
    # 计算评估指标
    accuracy = accuracy_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    # 打印评估结果
    print(f'\nEpoch {epoch+1}/{num_epochs}, Accuracy: {accuracy*100:.2f}%, Recall: {recall:.2f}, F1 Score: {f1:.2f}\n')

Epoch 1/80: 100%|██████████| 13/13 [00:01<00:00, 10.28batch/s, Loss=0.584]



Epoch 1/80, Accuracy: 52.00%, Recall: 0.52, F1 Score: 0.52



Epoch 2/80: 100%|██████████| 13/13 [00:01<00:00, 10.37batch/s, Loss=0.798]



Epoch 2/80, Accuracy: 43.50%, Recall: 0.43, F1 Score: 0.43



Epoch 3/80: 100%|██████████| 13/13 [00:01<00:00, 10.40batch/s, Loss=0.689]


...


Epoch 78/80, Accuracy: 50.50%, Recall: 0.51, F1 Score: 0.34



Epoch 79/80: 100%|██████████| 13/13 [00:01<00:00, 10.19batch/s, Loss=0.705]



Epoch 79/80, Accuracy: 51.50%, Recall: 0.52, F1 Score: 0.40



Epoch 80/80: 100%|██████████| 13/13 [00:01<00:00, 10.09batch/s, Loss=0.661]


Epoch 80/80, Accuracy: 48.00%, Recall: 0.48, F1 Score: 0.48

